<center> <h1> Assignment: Segmenting and Clustering Neighborhoods in Toronto </h1> </center>

**Preliminary note:** *this notebook will be developed throughout our capstone project which looks very exciting!*

## Importing libraries

*Before we get the data and start exploring it, let's download all the libraries that we will need.*

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Downloading and scraping the web page

We download the contents of the web page:

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
canada_data  = requests.get(url).text

We create the `soup` object

In [4]:
soup = BeautifulSoup(canada_data,"html.parser")

Let us find all the tables of the webpage.

In [5]:
tables = soup.find_all('table') # in html table is represented by the tag <table>

Now, we scrape the **postal codes table** in the desired format.

In [6]:
postal_code_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])
for row in tables[0].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        borough = col[1].text.strip()
        neighborhood = col[2].text.strip()
        postal_code = col[0].text.strip()
        
        postal_code_data = postal_code_data.append({"PostalCode":postal_code, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)

postal_code_data.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Preparing the postal codes dataframe

Now, we only deal with cells with an **assigned** borough. Therefore:

In [7]:
postal_code_data.drop(postal_code_data[postal_code_data['Borough']=='Not assigned'].index, inplace=True)
postal_code_data.reset_index(drop = True,inplace = True)
#postal_code_data.drop(['index'],axis=1, inplace=True)
postal_code_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Let us group the postal code dataset by boroughs

In [8]:
df_postcode = postal_code_data.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_postcode.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Let us find the **shape** of this dataframe:

In [9]:
df_postcode.shape

(103, 3)

There are 103 lines in the dataset!

## Obtaining the longitude and latitude of each neighborhood

In [10]:
print('We are going to read data from a .csv file')
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv',index_col='Postal Code')
geospatial_data.head()

We are going to read data from a .csv file


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Now, we **join** the two datasets to obtain the target dataset:

In [11]:
df_postcode_joined = df_postcode.join(geospatial_data, on='PostalCode')
df_postcode_joined.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Exploration of the neighborhoods in Toronto

In this section of the notebook, we finally explore and cluster the neighborhoods in Toronto, as we did for New York City during the hands-on lab sessions.

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_postcode_joined['Borough'].unique()),
        df_postcode_joined.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [13]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_postcode_joined['Latitude'], df_postcode_joined['Longitude'], df_postcode_joined['Borough'], df_postcode_joined['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'SZ4CTJ5PEJL3HVDEBANQ0QV2TGFTKFZBBLQ4WMOAXOBZQER3' # your Foursquare ID
CLIENT_SECRET = '4AMXTDALU1P4JQ0JG2MHJDC2GIJKYMIPDILGGD00B13ASWK2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = 'BZQJEB13MGEYT5WNOOFDIX2EFCWVHYB2N0VNKY0CNLCJSDXZ' # your FourSquare Access Token
print('My credentials will not be printed out, as they are personal!')

My credentials will not be printed out, as they are personal!


#### Let us create a function to get nearby venues to all the neighborhoods in Toronto¶

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, we can apply our function to the city of Toronto.

In [ ]:
toronto_venues = getNearbyVenues(names=df_postcode_joined['Neighborhood'],
                                   latitudes=df_postcode_joined['Latitude'],
                                   longitudes=df_postcode_joined['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East


In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

Let's check how many venues were returned for each neighborhood:

In [ ]:
toronto_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues¶

In [ ]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

## Analyze Toronto neighborhoods in detail

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#toronto_onehot = pd.get_dummies(don_mills[['Venue Category']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

The size of the new dataframe is

In [ ]:
toronto_onehot.shape

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

The new size is:

In [ ]:
toronto_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues¶

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Let's write a function to sort top venues in descending order:

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 5 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Neighborhoods clustering

We run the k-means algorithm to cluster the neighborhood into **5 clusters**.

In [ ]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init = 'k-means++', n_clusters=kclusters, n_init = 10, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_postcode_joined

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#Drop Nan values
toronto_merged.dropna(axis=0, inplace=True)

#toronto_merged.dropna(axis=0, inplace=True)
toronto_merged.head() # check the last columns!

Finally, let's vizualise the clusters:

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters